## Introduction


In this notebook, we use the Bluemix CLI tools to create a new IBM Analytics Engine instance that is configured to use IBM Cloud Object Storage (IBM COS).

*Prerequisites:* 
- You have worked through the notebook `examples/CLI/CLI_Setup.ipynb`

*Recommended:*
- You have worked through the notebook `examples/CLI/Provision_IAE.ipynb`

---

## Load utility library and set notebook width

To prevent this notebook from getting too cluttered, we use some python utilities.  We load them below.

In [ ]:
import sys
sys.path.append("../../modules")
import iae_examples

Let's set this notebook to use the full width of the browser using the utilities

In [ ]:
iae_examples.set_notebook_full_width()

---

## Read Cloud Foundry endpoint properties

We can read some variables saved when we ran the notebook `examples/CLI/CLI_Setup.ipynb` to configure our choosen api, org and space

In [ ]:
(CF_API, CF_ORG, CF_SPACE) = iae_examples.read_cf_target_endpoint_details('../../secrets/cf_target_endpoint.json')

---

## Save IBM Cloud Object Storage endpoint properties

Create a file `../../secrets/cos_s3_endpoint.json` with your COS credentials.  The file format should be:

```
{
   "S3_ACCESS_KEY":       "<AccessKey-changeme>",
   "S3_PRIVATE_ENDPOINT": "<Private-EndPoint-changeme>",
   "S3_PUBLIC_ENDPOINT":  "<Public-EndPoint-changeme>",
   "S3_SECRET_KEY":       "<SecretKey-changeme>"
}
```

Now let's load the cos file into some variables that we will use later

In [ ]:
(S3_ACCESS_KEY, S3_PRIVATE_ENDPOINT, S3_PUBLIC_ENDPOINT, S3_SECRET_KEY) = \
    iae_examples.read_cos_endpoint_details('../../secrets/cos_s3_endpoint.json')

---

## Upload file to COS

In [ ]:
! pip install --upgrade boto

In [ ]:
import boto
import boto.s3.connection
import logging

boto.set_stream_logger('paws', level=logging.ERROR)

conn = boto.connect_s3(
        aws_access_key_id = S3_ACCESS_KEY,
        aws_secret_access_key = S3_SECRET_KEY,
        host = S3_PUBLIC_ENDPOINT,
        calling_format = boto.s3.connection.OrdinaryCallingFormat(),
        )

In [ ]:
try:
    bucket = conn.get_bucket('temp-bucket')
except:
    bucket = conn.create_bucket('temp-bucket')
    
print(bucket.name)

In [ ]:
! wget -q https://raw.githubusercontent.com/snowch/IBM_Analytics_Engine_Examples/master/scripts/COS_S3.sh

In [ ]:
key = bucket.new_key('COS_S3_bootstrap.sh')
key.set_contents_from_filename('COS_S3.sh')

---

## Provision IAE instance

Before we can provision IAE, we need to login to Bluemix using the Bluemix CLI

In [ ]:
! bx login --apikey @../../secrets/apiKey.json -a {CF_API} -o {CF_ORG} -s {CF_SPACE}

There are a few ways to configure IAE to use IBM COS.   Let's automate the process with a custom script.

**NOTE:** These examples prefer automation to manual approaches for configuration.  One key benefit of automation is that it supports creating environments in a repeatable and testable way.

In [ ]:
import json

custom_script = { 
    "num_compute_nodes": 1, 
    "hardware_config": "Standard", 
    "software_package": "ae-1.0-spark",
    "customization": [{
        "name": "action1",
        "type": "bootstrap",
        "script": {
            "source_type": "CosS3",
            "source_props": {
                "auth_endpoint": S3_PRIVATE_ENDPOINT,
                "access_key_id": S3_ACCESS_KEY,
                "secret_access_key": S3_SECRET_KEY
             },
         "script_path": bucket.name + "/COS_S3_bootstrap.sh"
        },
        "script_params": [S3_ACCESS_KEY, S3_PRIVATE_ENDPOINT, S3_SECRET_KEY]
    }]
}

# write the script to a file in the local directory where we can access it in the next step using the Bluemix CLI 

with open('../../secrets/custom_script.json', 'w') as f:
    f.write(json.dumps(custom_script))

We can then attempt to create an IBM Analytics Engine Instance.

In [ ]:
! bx cf create-service IBMAnalyticsEngine Standard 'myiaeinstance' -c ../../secrets/custom_script.json

---

Note the output from above.  If all went ok, the CLI should suggest running `cf service myiaeinstance` to check the provisioning status. Let's do that now.

**NOTE:** If there is an error output by the above step, jump to the section below on debugging.



In [ ]:
! bx cf service myiaeinstance

---

## Create service key

Here we create a service key which contains the cluster credentials. 
We export the service key information to a file. 
We can then read the service key details into python variables so we can use those variables later in this notebook.

In [ ]:
! bx cf create-service-key myiaeinstance myiaeinstance_servicekey

In [ ]:
! bx cf service-keys myiaeinstance

In [ ]:
! bx cf service-key myiaeinstance myiaeinstance_servicekey > ../../secrets/iae_service_key.json

# unfortunately, the output of this command contains some lines of text before the json
# lets remove the first four lines of output and save the raw json

# TODO move this to the utility module

with open('../../secrets/iae_service_key.json') as data_file:
    data = ''.join(data_file.readlines()[4:])
    
with open('../../secrets/iae_service_key.json', 'w') as data_file:
    data_file.write(data)

In [ ]:
# TODO move this to the utility module

import json

with open('../../secrets/iae_service_key.json') as data_file:    
    iae_service_key = json.load(data_file)
    
livy   = iae_service_key['cluster']['service_endpoints']['livy']
ambari = iae_service_key['cluster']['service_endpoints']['ambari_console']
user   = iae_service_key['cluster']['user']
pswd   = iae_service_key['cluster']['password']
   
print('livy:   '   + livy)
print('ambari: '   + ambari)
print('user:   '   + user)

---

## Verify COS was successfully configured

In [ ]:
# TODO - this doesn't detect the properties even when they are set ok

iae_examples.is_s3_access_key_set(ambari, user, pswd, S3_ACCESS_KEY)

---

## Upload spark script to COS

In [ ]:
file_contents = """
from __future__ import print_function

import sys
from random import random
from operator import add

from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession.builder.appName("PythonPi").getOrCreate()

    partitions = 2
    n = 100000 * partitions

    def f(_):
        x = random() * 2 - 1
        y = random() * 2 - 1
        return 1 if x ** 2 + y ** 2 <= 1 else 0

    count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
    print("Pi is roughly %f" % (4.0 * count / n))

    spark.stop()
"""

with open('PiEx.py', 'w') as pyspark_file:
    pyspark_file.write(file_contents)

In [ ]:
key = bucket.new_key('PiEx.py')
key.set_contents_from_filename('PiEx.py')

---

## Analyse data with Spark 

Execute spark job

In [ ]:
import requests

headers = { 
    'Content-Type': 'application/json',
    'X-Requested-By': 'livy'
}
data = { "file":"s3a://{0}/PiEx.py".format(bucket.name) }

res = requests.post(livy, auth=(user, pswd), headers=headers, data=json.dumps(data))
print(res.text)

id = res.json()['id']

Get job state

In [ ]:
headers = { 
    'Content-Type': 'application/json',
    'X-Requested-By': 'livy'
}
url = '{0}/{1}'.format(livy, id)
response = requests.get(url, auth=(user, pswd), headers=headers)
print(response.json()['state'])

Let's take a look at the spark job log

In [ ]:
headers = { 
    'Content-Type': 'application/json',
    'X-Requested-By': 'livy'
}
url = '{0}/{1}/log'.format(livy, id)
response = requests.get(url, auth=(user, pswd), headers=headers)
print('\n'.join(response.json()['log']))

---

## Debugging 

TODO 

In [ ]:
! bx cf space dev --guid

In [ ]:
! bx cf services

In [ ]:
! bx cf service-keys myiaeinstance

In [ ]:
! bx cf delete-service-key myiaeinstance myiaeinstance_servicekey -f

In [ ]:
! bx cf delete-service myiaeinstance -f 